In [1]:
!pip install transformers
!pip install av

  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---------------------------------------- 10.4/10.4 MB 107.7 MB/s eta 0:00:00
Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl (162 kB)
Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl (274 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 135.8 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
   ---------------------------------------- 0.0/30.9 MB ? eta -:--:--
   ------------------- ---------

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import tqdm
import av
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [2]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# os.mkdir('results')

torch.manual_seed(0)
np.random.seed(0)

In [ ]:
# Download UCF-101 dataset and labels
# Download data
# !curl -L -o UCF101.rar https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
# !unrar x UCF101.rar
# !rm UCF101.rar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 6611M    0 7711k    0     0  12.6M      0  0:08:41 --:--:--  0:08:41 12.6M
  0 6611M    0 37.3M    0     0  23.4M      0  0:04:41  0:00:01  0:04:40 23.5M
  1 6611M    1 80.8M    0     0  31.2M      0  0:03:31  0:00:02  0:03:29 31.2M
  2 6611M    2  136M    0     0  38.0M      0  0:02:53  0:00:03  0:02:50 38.1M
  2 6611M    2  187M    0     0  40.8M      0  0:02:41  0:00:04  0:02:37 40.8M
  3 6611M    3  222M    0     0  39.8M      0  0:02:45  0:00:05  0:02:40 43.0M
  3 6611M    3  259M    0     0  39.4M      0  0:02:47  0:00:06  0:02:41 44.5M
  4 6611M    4  298M    0     0  39.3M      0  0:02:48  0:00:07  0:02:41 43.5M
  5 6611M    5  338M    0     0  39.3M      0  0:02:47  0:00:08  0:02:39 40.2M
  5 6611M    5  378M    0     0  39.4M      0  0:02

In [3]:
# Download train & test split
# !curl -L https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip -O UCF101TrainTestSplits-RecognitionTask.zip
# !unzip -q UCF101TrainTestSplits-RecognitionTask.zip
# !rm UCF101TrainTestSplits-RecognitionTask.zip

In [4]:
# !copy /b ./ucfTrainTestlist/testlist01.txt + ./ucfTrainTestlist/testlist02.txt + ./ucfTrainTestlist/testlist03.txt ./ucfTrainTestlist/testlist.txt
# !copy /b ./ucfTrainTestlist/trainlist01.txt + ./ucfTrainTestlist/trainlist02.txt + ./ucfTrainTestlist/trainlist03.txt ./ucfTrainTestlist/trainlist.txt

In [3]:
UCF_CLASSES = ['ApplyEyeMakeup','ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'Hammering', 'HammerThrow', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpingJack', 'JumpRope', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'PlayingViolin', 'PoleVault', 'PommelHorse', 'PullUps', 'Punch', 'PushUps', 'Rafting', 'RockClimbingIndoor', 'RopeClimbing', 'Rowing', 'SalsaSpin', 'ShavingBeard', 'Shotput', 'SkateBoarding', 'Skiing', 'Skijet', 'SkyDiving', 'SoccerJuggling', 'SoccerPenalty', 'StillRings', 'SumoWrestling', 'Surfing', 'Swing', 'TableTennisShot', 'TaiChi', 'TennisSwing', 'ThrowDiscus', 'TrampolineJumping', 'Typing', 'UnevenBars', 'VolleyballSpiking', 'WalkingWithDog', 'WallPushups', 'WritingOnBoard', 'YoYo']

In [4]:
ucf_train_df = pd.read_csv('ucfTrainTestlist/trainlist.txt', sep=' ', header=None)
ucf_train_df.columns = ['id', 'label']

ucf_valid_df = ucf_train_df.sample(frac=0.2)
ucf_valid_df['id'] = ucf_valid_df['id'].apply(lambda x: f"./UCF-101/UCF-101/{x}")

ucf_train_df = ucf_train_df.drop(ucf_valid_df.index)
ucf_train_df['id'] = ucf_train_df['id'].apply(lambda x: f"./UCF-101/UCF-101/{x}")
ucf_train_df['label'] = ucf_train_df['label'].apply(lambda x: x-1)
ucf_valid_df['label'] = ucf_valid_df['label'].apply(lambda x: x-1)

print(ucf_train_df.head())
print("Number of rows : ", ucf_train_df.shape[0])

                                                  id  label
0  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
1  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
2  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
3  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
5  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
Number of rows :  22998


In [5]:
ucf_class_df = pd.read_csv('ucfTrainTestlist/classInd.txt', sep=' ', header=None)
ucf_class_df.columns = ['label', 'label_name']

ucf_test_df = pd.read_csv('ucfTrainTestlist/testlist.txt', sep=' ', header=None)
ucf_test_df.columns = ['id']
ucf_test_df['label'] = ucf_test_df['id'].str.split('/').str[0]

label_mapping = dict(zip(ucf_class_df['label_name'], ucf_class_df['label']))
ucf_test_df['label'] = ucf_test_df['label'].map(label_mapping)

ucf_test_label_df = ucf_test_df[['id', 'label']]
ucf_test_df = ucf_test_df.drop(columns=['label'])

ucf_test_label_df['label'] = ucf_test_label_df['label'].apply(lambda x: x-1)
ucf_test_label_df['id'] = ucf_test_label_df['id'].apply(lambda x: f"./UCF-101/UCF-101/{x}")

# N_CALL_UCF = ucf_test_df['label'].nunique()

print(ucf_test_label_df.head())
print("Number of rows : ", ucf_test_label_df.shape[0])

                                                  id  label
0  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
1  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
2  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
3  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
4  ./UCF-101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMak...      0
Number of rows :  11213


In [6]:
def read_video_pyav(container, indices):
    '''
    ...     Decode the video with PyAV decoder.
    ...     Args:
    ...         container (`av.container.input.InputContainer`): PyAV container.
    ...         indices (`List[int]`): List of frame indices to decode.
    ...     Returns:
    ...         result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    ...     '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)

    if len(frames) == 0 :
        pass

    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [7]:
def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
        ...     Sample a given number of frame indices from the video.
        ...     Args:
        ...         clip_len (`int`): Total number of frames to sample.
        ...         frame_sample_rate (`int`): Sample every n-th frame.
        ...         seg_len (`int`): Maximum allowed index of sample's last frame.
        ...     Returns:
        ...         indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)

    while converted_len >= seg_len:
        # You could either adjust clip_len or frame_sample_rate, or both
        # For example, reduce clip_len to fit the available frames:
        frame_sample_rate = seg_len // clip_len
        # Recalculate converted_len based on the adjusted clip_len
        converted_len = clip_len * frame_sample_rate


        if converted_len == seg_len:
            frame_sample_rate -= 1
            converted_len = clip_len * frame_sample_rate

    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [8]:
def format_video(video_path):
    container = av.open(video_path)
    seg_len = int(container.streams.video[0].frames)
    indices = sample_frame_indices(clip_len=8, frame_sample_rate=4, seg_len=seg_len)
    video = read_video_pyav(container, indices)
    return video

In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, df):
        """
        Args:
            image_data (list or np.array): Preprocessed image data, should be in shape (num_samples, height, width, channels).
            labels (list or np.array): Labels corresponding to the images.
        """
        self.df = df

    def __len__(self):
        # Return the total number of samples
        return self.df.shape[0]

    def __getitem__(self, idx):
        # Retrieve the image and label at index `idx`
        row = self.df.iloc[idx]
        image = row['id']
        label = int(row['label'])

        # If your image needs to be converted to a torch tensor
        # image = torch.tensor(image, dtype=torch.float32)  # Adjust dtype if necessary

        # Depending on your label format, convert the label
        label = torch.tensor(label, dtype=torch.long)  # Assuming it's a classification problem

        return image, label

In [10]:
data = [ucf_train_df, ucf_valid_df, ucf_test_label_df]

# data = [train_df, validation_df, test_label_df]

train_dataset = CustomImageDataset(data[0])
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

valisation_dataset = CustomImageDataset(data[1])
val_loader = DataLoader(valisation_dataset, batch_size=1, shuffle=True)

evaluation_dataset = CustomImageDataset(data[2])
evaluation_loader = DataLoader(evaluation_dataset, batch_size=1, shuffle=True)

In [11]:
def evaluation_run(model, image_processor, criterion, evaluation_set):
    model.eval()

    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0

        for vid_id, labels in tqdm.tqdm(evaluation_set):
            vid_id = vid_id[0]

            try:
                images = format_video(vid_id)

                images = torch.tensor(images, dtype=torch.float32)

                images = torch.squeeze(images)
                inputs = image_processor(list(images), return_tensors="pt")
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)


                outputs = model(**inputs)
                logits = outputs['logits']

            except Exception as e:
                total += 1 
                continue


            running_loss += criterion(logits, labels)
            _, predicted = torch.max(logits, 1)
            total += 1
            correct += (predicted == labels).sum().item()

    validation_loss = running_loss / len(evaluation_set)
    accuracy = (100 * correct) / total
    print(f"Evaluation : Loss: {validation_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return validation_loss

In [ ]:
BATCH_SIZE = 16

def validation_run(model, image_processor, criterion, validation_set):
    model.eval()

    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0

        for vid_id, labels in tqdm.tqdm(validation_set):
            vid_id = vid_id[0]

            try:
                images = format_video(vid_id)

                images = torch.tensor(images, dtype=torch.float32)

                images = torch.squeeze(images)
                inputs = image_processor(list(images), return_tensors="pt")
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)


                outputs = model(**inputs)
                logits = outputs['logits']
            except Exception as e:
                continue

            running_loss += criterion(logits, labels)
            _, predicted = torch.max(logits, 1)
            total += 1
            correct += (predicted == labels).sum().item()

    validation_loss = running_loss / len(validation_set)
    accuracy = (100 * correct) / total
    print(f"Validation : Loss: {validation_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return validation_loss

def train_model(model, image_processor, training_dataloader, criterion, optimizer, num_epochs=10, validation_dataloader=val_loader):
    validation_loss = []
    training_loss = []

    for epoch in range(num_epochs):
        i = 1
        running_loss = 0.0
        correct = 0
        total = 0

        loss = 0
        model.train(True)

        for vid_id, labels in tqdm.tqdm(training_dataloader):
            vid_id = vid_id[0]

            try:
                images = format_video(vid_id)

                images = torch.tensor(images, dtype=torch.float32)
                images = torch.squeeze(images)

                inputs = image_processor(list(images), return_tensors="pt")
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                outputs = model(**inputs)
                logits = outputs['logits']
            except Exception as e:
                total += 1
                torch.cuda.empty_cache()
                # print(e)
                continue
            
            # Calculer la perte
            loss = criterion(logits, labels) / BATCH_SIZE
            loss.backward()

            # Rétropropagation de la perte
            if (i+1) % BATCH_SIZE == 0:
              # Mettre à jour les paramètres du modèle
              optimizer.step()
              optimizer.zero_grad()

            i += 1

            # Calcul des statistiques
            running_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            total += 1
            try:
                correct += (predicted == labels).sum().item()
            except Exception as e:
                continue
        
        optimizer.step()
        optimizer.zero_grad()
        
        # Affichage des statistiques après chaque époque
        epoch_loss = running_loss / len(training_dataloader)
        accuracy = (100 * correct) / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")
        val_loss = validation_run(model, image_processor, criterion, validation_dataloader)

        validation_loss.append(val_loss)
        training_loss.append(epoch_loss)

        torch.save(model.state_dict(), f"./results/training_{TRAINING}/weights_epoch%d.pt"%epoch)

    return training_loss, validation_loss

In [13]:
import sys
sys.modules.pop('implementations.llora_timesformer', None)

In [14]:
from transformers import AutoImageProcessor

import importlib
llora_mod = importlib.import_module('implementations.prefix_timesformer')
importlib.reload(llora_mod)
TimesformerForVideoClassification = llora_mod.TimesformerForVideoClassification

TRAINING = 'PREFIX' # Modify this value each run
# os.mkdir(f"./results/training_{TRAINING}")

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400", num_labels=len(UCF_CLASSES)+1, ignore_mismatched_sizes=True)
model.train(True)
model.to(DEVICE)

c:\Users\calorrain\AppData\Local\anaconda3\envs\visu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of TimesformerForVideoClassification were not initialized from the model checkpoint at facebook/timesformer-base-finetuned-k400 and are newly initialized: ['timesformer.encoder.layer.0.attention.attention.prefix_k.embeddings.bias', 'timesformer.encoder.layer.0.attention.attention.prefix_k.embeddings.weight', 'timesformer.encoder.layer.0.attention.attention.prefix_v.embeddings.bias', '

TimesformerForVideoClassification(
  (timesformer): TimesformerModel(
    (embeddings): TimesformerEmbeddings(
      (patch_embeddings): TimesformerPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (time_drop): Dropout(p=0.0, inplace=False)
    )
    (encoder): TimesformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x TimesformerLayer(
          (drop_path): Identity()
          (attention): TimeSformerAttention(
            (attention): TimesformerSelfAttention(
              (qkv): Linear(in_features=768, out_features=2304, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (prefix_k): PrefixModule(
                (embeddings): Linear(in_features=768, out_features=768, bias=True)
              )
              (prefix_v): PrefixModule(
                (embeddings): Linear(in_features=768, out_features=768, bias=True)
              )
       

In [15]:
for name, param in model.named_parameters():
    if 'prefix' not in name and 'classifier' not in name:
        param.requires_grad = False
    else :
        print(name)

timesformer.encoder.layer.0.attention.attention.prefix_k.embeddings.weight
timesformer.encoder.layer.0.attention.attention.prefix_k.embeddings.bias
timesformer.encoder.layer.0.attention.attention.prefix_v.embeddings.weight
timesformer.encoder.layer.0.attention.attention.prefix_v.embeddings.bias
timesformer.encoder.layer.0.temporal_attention.attention.prefix_k.embeddings.weight
timesformer.encoder.layer.0.temporal_attention.attention.prefix_k.embeddings.bias
timesformer.encoder.layer.0.temporal_attention.attention.prefix_v.embeddings.weight
timesformer.encoder.layer.0.temporal_attention.attention.prefix_v.embeddings.bias
timesformer.encoder.layer.1.attention.attention.prefix_k.embeddings.weight
timesformer.encoder.layer.1.attention.attention.prefix_k.embeddings.bias
timesformer.encoder.layer.1.attention.attention.prefix_v.embeddings.weight
timesformer.encoder.layer.1.attention.attention.prefix_v.embeddings.bias
timesformer.encoder.layer.1.temporal_attention.attention.prefix_k.embeddings

In [16]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

print(params)

28426854


In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Set the scheduler to decay the LR by 10x at epochs 11 and 14
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[11, 14], gamma=0.1)

In [18]:
validation_loss, training_loss = train_model(model, image_processor, train_loader, criterion, optimizer, num_epochs=15)

100%|██████████| 22998/22998 [48:22<00:00,  7.92it/s]


Epoch [1/15], Loss: 0.0532, Accuracy: 86.58%


100%|██████████| 5749/5749 [07:12<00:00, 13.28it/s]


Validation : Loss: 0.0764, Accuracy: 99.01%


100%|██████████| 22998/22998 [48:24<00:00,  7.92it/s]


...

Validation : Loss: 0.0082, Accuracy: 99.81%


100%|██████████| 22998/22998 [49:12<00:00,  7.79it/s]


Epoch [15/15], Loss: 0.0002, Accuracy: 99.94%


100%|██████████| 5749/5749 [07:32<00:00, 12.70it/s]


Validation : Loss: 0.0033, Accuracy: 99.91%


In [19]:
evaluation_loss = evaluation_run(model, image_processor, criterion, evaluation_loader)

100%|██████████| 11213/11213 [14:30<00:00, 12.87it/s]

Evaluation : Loss: 0.0017, Accuracy: 99.93%
